In [ ]:
"""
##############
# 크롤링 리스트 #
########################################################
광주 아파트, 광주 학교, 광주 빌라, 광주 교회, 광주 성당, 광주 빌딩, 
광주 유치원, 광주 초등학교, 광주 중학교, 광주 고등학교, 광주 대학교, 
광주 음식점, 광주 카페, 광주 시장, 광주 공원    
########################################################
"""

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm_notebook

In [ ]:
driver=webdriver.Chrome("/Users/jinwon-kim/Developer/chromedriver")

In [ ]:
url="https://map.kakao.com/"
driver.get(url)
time.sleep(5)

driver.find_element_by_xpath('//*[@id="search.keyword.query"]').click()
# search=driver.find_element_by_css_selector('#search\.keyword\.query')
search=driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
search.send_keys("광주광역시 아파트")
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').click()
driver.find_element_by_xpath('//*[@id="info.search.place.more"]').click() # 더보기 최초 1회

In [ ]:
html=driver.page_source
soup=BeautifulSoup(html, 'html.parser')

loc_list = soup.select("div > ul.placelist > li > div.head_item.clickArea")
addrs=soup.select("#info\.search\.place\.list > li:nth-child(n)")
# sub_addrs=soup.select("#info\.search\.place\.list > li:nth-child(n)")
# div.head_item.clickArea # 아파트명
# div.info_item # 주소
# div.rating.clickArea # 평점
# div.DupListView > ul # 몇 동 101동 등등
# #info\.search\.place\.list > li:nth-child(n) 여기에 주소 및 숨은 동 까지 다 들어 있다
# <ul class="duplist HIDDEN" data-id="duplist"><li class="DupItemView"> # 이게 있을경우 101동~ 등등 이 있다

In [ ]:
# 함수로 만들기
tmp_subadrr = []
tmp = []
searched_info={'Name':[],'SubAddr':[],'Category':[],'Address':[],'OtherAddr':[]}
for i, addr in enumerate(addrs):
    order = i + 1
    # print(len(addr.select("div.DupListView > ul")))
    if len(addr.select("div.DupListView > ul"))>0:
        # 이름
        print(addr.select("li:nth-child({}) > div.utile_item > span.screen_out".format(order))[0].text.strip())
        name = addr.select("li:nth-child({}) > div.utile_item > span.screen_out".format(order))[0].text.strip()
        # 동 (있을경우)
        # print(addr.select("li:nth-child({}) > div.DupListView > ul".format(order))[0].text.strip())
        tmp_subadrr.append(addr.select("li:nth-child({}) > div.DupListView > ul".format(order))[0].text.strip())
        # print(tmp_subadrr)
        tmp_subadrr = tmp_subadrr[0].split("\n")
        tmp_subadrr = [i for i in tmp_subadrr if i != '']
        print(tmp_subadrr)
        subAddr = tmp_subadrr
        # 타입-category (아파트, 식당)
        print(addr.select("li:nth-child({}) > div.head_item.clickArea > strong > a.link_name > strong".format(order))[0].text.strip())
        category = addr.select("li:nth-child({}) > div.head_item.clickArea > strong > a.link_name > strong".format(order))[0].text.strip()
        # 주소
        print(addr.select("li:nth-child({}) > div.info_item > div.addr > p:nth-child(1)".format(order))[0].text.strip())
        address = addr.select("li:nth-child({}) > div.info_item > div.addr > p:nth-child(1)".format(order))[0].text.strip()
        # 주소(지번)
        print(addr.select("li:nth-child({}) > div.info_item > div.addr > p.lot_number".format(order))[0].text.strip())
        otherAddr = addr.select("li:nth-child({}) > div.info_item > div.addr > p.lot_number".format(order))[0].text.strip()

        searched_info['Name'].append(name)
        searched_info['SubAddr'].append(subAddr)
        searched_info['Category'].append(category)
        searched_info['Address'].append(address)
        searched_info['OtherAddr'].append(otherAddr)
    tmp_subadrr = []
    
    
# try except 로 없는것들 생길수도 있는것 예외처리할것
    # 광고는 안들어감
    # 그리고 주소 동 있을때 넣는 함수 필요
    
    # if len(addr.select("li:nth-child({}) > div.DupListView > ul".format(order)))>0:
        # print(addr.select("li:nth-child({}) > div.DupListView > ul".format(order)))
        # print(addr.select("div > strong > a.link_name")[0].text.strip())
        # tmp.append(addr.select("li:nth-child({}) > div.DupListView > ul".format(order)))
        # #info\.search\.place\.list > li:nth-child(10) > div.DupListView > ul
        # text = f'{i}'
        # print(text)
    # else: #이것은 광고였다
    #     text = f'{i}_None'
    #     tmp.append(None)
    #     print(f'{i}_None')

# #info\.search\.place\.list > li:nth-child(1) > div.DupListView
#info\.search\.place\.list
#info\.search\.place\.list > li:nth-child(1) > div.utile_item > span #집이름
#info\.search\.place\.list > li:nth-child(1) > div.head_item.clickArea > strong > a.link_name
#info\.search\.place\.list > li.AdItem > a > div.info_ad > strong > span:nth-child(1) #(광고)

In [ ]:
pd.DataFrame(searched_info)

In [ ]:
# 함수 예시 
def get_store_addr(addrs):
    addr_info={'Address':[], 'Hour':[], 'Phone':[]}
    for i, addr in enumerate(addrs):
        try:
            location=addr.select("div.addr > p")[0].text.strip()
            hour=addr.select("div.openhour > p > a")[0].text.strip()
            phone=addr.select("div.contact.clickArea > span.phone")[0].text.strip()
        except:
            location=""
            hour=""
            phone=""
        
        # 딕셔너리 업데이트
        addr_info['Address'].append(location)
        addr_info['Hour'].append(hour)
        addr_info['Phone'].append(phone)
    return addr_info